<a href="https://colab.research.google.com/github/nvvishnu/CS6910-Assignments/blob/main/Assignement1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb
import pandas as pd
import numpy as np
import wandb
from keras.datasets import fashion_mnist
from matplotlib import pyplot
import math

In [ ]:
(trainX, trainy), (testX, testy) = fashion_mnist.load_data()

In [ ]:
np.unique(trainy)
trainy.T.shape

(60000,)

In [ ]:
def softmax(arr):
    
    l = []

    flag = False

    cnt = 0
  
    y = []
    for i in range(arr.shape[0]) : 
      
      if(arr[i][0]<=600):
         l.append(math.exp(arr[i][0]))
      else:
        flag = True
        cnt = cnt + 1
        y.append(i)
#      print(i)

    if(flag):
       
      # print('array length in softmax',len(arr))
       temp = np.zeros(len(arr))
    
       for it in y:
       #   print(type(it),type(temp))
          temp[it] = 1.0/cnt;

       temp = temp.reshape(temp.shape[0],1) 
          
       return temp   
         
    if(np.sum(l) != 0):

      l = np.array(l)
      l = (l)/np.sum(l)

      l = l.reshape((l.shape[0],1))

      return l

    else:
       return (1.0/len(arr))*(np.ones(shape=(len(arr),1),dtype=float))


def feed_forward(X,activation_function,weights,biases,output_function=softmax):
    
    X = X.copy()
    X = X.flatten()
    num_hidden_layers = len(biases)
    # X is the input (x1,x2,...,xn)
    # activation_function is just the activation function s.t h_i = activation_function(a_i)
    # weights is a list of m*n where n is number of neurons in previous layer and m is number of neurons in current layer assume 1 based indexing 
    # assuming output function O is softmax function
    # biases is list of  [b1,b2,..,bk] for each layer

    X = X.reshape((X.shape[0],1))
    a = []
    h = []
    
    for i in range(num_hidden_layers):
       if(i == 0):
         a.append(np.array(0))
         h.append(X)
       else:
        # a_i = W_ih_(i-1) + b_i
        #print(weights[i].shape,h[i-1].shape,biases[i].shape)
        # print(weights[i],h[i-1])
        a.append(np.matmul(weights[i],h[i-1])+biases[i])
        #print(a[len(a)-1].shape)
        h.append(activation_function(a[len(a)-1]))
    # print("a is ")
    # for it in a:
    #    print(it.flatten())
    # print("####")
    # print("h is ")
    # for i in range(4):
    #    if(i != 0):
    #       print(h[i].flatten())    
    return [a,h,output_function(a[len(a)-1])]
    

In [ ]:
def calculate_gradients(y_true,y_out,h,a,weights,act_fn_derivative):
    
    #delta(A_l) = -(e_ytrue - y_out)
    #delta(h_i) = (W_i+1).Tdelta(A_i+1)  (for i = L-1 to 1)
    #delta(A_i) = delta(h_i)*(activation_function'(A_i)) for(i = L-1 to 1)
    #delta(W_k) = delta(A_k)(h_(k-1).T) for(i = L to i = 1)
    #delta(b_k) = delta(A_k)

    e = np.zeros(len(y_out))
    e = e.reshape(e.shape[0],1)
    e[y_true] = 1
    A = y_out-e
    l = len(h)
    delta_W = []
    delta_B = []
    
    delta_W.append(np.matmul(A,h[l-2].T))
#    print(delta_W[0].shape,weights[l-1].shape)
    delta_B.append(A)
    l = l-2

    while(l >= 1):
       H = np.matmul(weights[l+1].T,A)
       A = H*(act_fn_derivative(a[l]))     # assuming a is 1 based index 
       delta_W.append(np.matmul(A,h[l-1].T))
 #      print(delta_W[len(delta_W)-1].shape,weights[l].shape)
       delta_B.append(A)
       l = l-1
    
    delta_W.append(0)
    delta_B.append(0)
    delta_W.reverse()
    delta_B.reverse()
    return [delta_W,delta_B]

    



In [ ]:
def upd_lst(lst1,lst2,operation,eeta): #Performs x 'operation' eeta*y for each element x in lst1 and y in lst2. 'operation' - addition/subtraction
   l = []
   if(len(lst1)!=len(lst2)):
    print("Lists are not of equal length")
 #  print(len(lst1),len(lst2))
   for (x,y) in zip(lst1,lst2):
   #  if(type(x) != int and type(y) !=int):
   #     print('....',x.shape,y.shape)
     if(operation == '+'):
       l.append(x+(eeta*y))
     else:
       l.append(x-(eeta*y))

   return l

In [ ]:
def sgd(X,Y,num_epochs,weight_decay,learning_rate,batch_size,weights,biases,activation_function,act_fn_derivative):
    
    for i in range(num_epochs): 
          cnt = 0
          cum_W = [np.zeros(weights[i].shape) if(i != 0) else 0 for i in range(len(weights))]
          cum_B = [np.zeros(biases[i].shape) if(i != 0) else 0 for i in range(len(biases))]
          for j in range(X.shape[0]):

            cnt = cnt + 1
            r = feed_forward(X[j],activation_function,weights,biases)
            a = r[0]
            h = r[1]
            yout = r[2]
            # print(yout)
            # if(cnt == 2):
            #    break
            gradients = calculate_gradients(Y[j],yout,h,a,weights,act_fn_derivative)
            cum_W = upd_lst(cum_W,gradients[0],'+',1)
            cum_B = upd_lst(cum_B,gradients[1],'+',1)
            
            if(cnt == batch_size): 
                weights = upd_lst(weights,cum_W,'-',(learning_rate*(1.0))/batch_size)
                biases = upd_lst(biases,cum_B,'-',(learning_rate*(1.0))/batch_size)
                cum_W = [np.zeros(weights[i].shape) if(i != 0) else 0 for i in range(len(weights))]
                cum_B = [np.zeros(biases[i].shape) if(i != 0) else 0 for i in range(len(biases))]
                cnt = 0
          print(i)
    return [weights,biases]

In [ ]:
def momentum(X,Y,num_epochs,weight_decay,learning_rate,batch_size,weights,biases,activation_function,act_fn_derivative):
  #update{t} = gamma . update_{t-1} + learning_rate * grad(weight)
  #weigt{t}=weight{t-1}-update{t}
  gamma=0.9 # The typical value of gamma used is 0.9
  for lmao in range(num_epochs):
    #Initialisation of weights_grad_tot, biases_grad_tot
    weights_grad_tot = [np.zeros(weights[i].shape) if(i != 0) else 0 for i in range(len(weights))]
    biases_grad_tot = [np.zeros(biases[i].shape) if(i != 0) else 0 for i in range(len(biases))]

    #Initialisation of update_weights
    update_weights=weights_grad_tot
    update_biases=biases_grad_tot

    print(lmao)
    # num_hidden_layers=len(biases) #Biases is a list of 1D arrays. Thus, the number of hidden layers is given by the size of this list    
    num_data_point_done=0
    dum=0
    for x,y in zip(X,Y):  
      #print(dum)
      #dum=dum+1
      y_pred=feed_forward(x,activation_function,weights,biases)  #Find prediction for each data instance of the data set
      a = y_pred[0]
      h = y_pred[1]
      yout = y_pred[2]
      [weights_grad,biases_grad] =  calculate_gradients(y,yout,h,a,weights,act_fn_derivative)
      weights_grad_tot = upd_lst(weights_grad_tot,weights_grad,'+',1)
      biases_grad_tot = upd_lst(biases_grad_tot,biases_grad,'+',1)
      num_data_point_done=num_data_point_done+1
      if(num_data_point_done==batch_size):
        update_weights = upd_lst(update_weights,update_weights,'+',gamma-1)
        update_biases = upd_lst(update_biases,update_biases,'+',gamma-1)
        update_weights=upd_lst(update_weights,weights_grad_tot,'+',learning_rate/batch_size)
        update_biases=upd_lst(update_biases,biases_grad_tot,'+',learning_rate/batch_size)
        # update_weights = update_weights + gamma*update_weights + learning_rate * weights_grad_tot
        # update_biases = update_biases + gamma*update_weight + learning_rate * biases_grad_tot
        weights=upd_lst(weights,update_weights,'-',1.0)
        biases=upd_lst(biases,update_biases,'-',1.0)
        num_data_point_done=0
        weights_grad_tot = [np.zeros(weights[i].shape) if(i != 0) else 0 for i in range(len(weights))]
        biases_grad_tot = [np.zeros(biases[i].shape) if(i != 0) else 0 for i in range(len(biases))]
  return [weights,biases]
  
def nesterov(X,Y,num_epochs,weight_decay,learning_rate,batch_size,weights,biases,activation_function,act_fn_derivative):
  #w_ahead=w_{t} - gamma. update_{t-1}
  #update{t} = gamma . update_{t-1} + learning_rate * grad(w_lookahead)

  gamma=0.9 # The typical value of gamma used is 0.9
  #Initialisation of weights_grad_tot, biases_grad_tot
  weights_grad_tot = [np.zeros(weights[i].shape) if(i != 0) else 0 for i in range(len(weights))]
  biases_grad_tot = [np.zeros(biases[i].shape) if(i != 0) else 0 for i in range(len(biases))]

  #Initialisation of update_weights
  update_weights=weights_grad_tot
  update_biases=biases_grad_tot  
  for lmao in range(num_epochs):
    print(lmao)
    # num_hidden_layers=len(biases) #Biases is a list of 1D arrays. Thus, the number of hidden layers is given by the size of this list    
    num_data_point_done=0
    for x,y in zip(X,Y):  
      y_pred=feed_forward(x,activation_function,weights,biases)  #Find prediction for each data instance of the data set
      a = y_pred[0]
      h = y_pred[1]
      yout = y_pred[2]
      [weights_grad,biases_grad] =  calculate_gradients(y,yout,h,a,w_ahead,act_fn_derivative)
      weights_grad_tot = upd_lst(weights_grad_tot,weights_grad,'+',1)
      biases_grad_tot = upd_lst(biases_grad_tot,biases_grad,'+',1)   
      num_data_point_done=num_data_point_done+1
      if(num_data_point_done==batch_size):
        w_ahead=upd_lst(weights,update_weights,'-',1)
        b_ahead=upd_lst(biases,update_biases,'-',1)
        update_weights = upd_lst(update_weights,update_weights,'+',gamma)
        update_biases = upd_lst(update_biases,update_biases,'+',gamma)
        update_weights=upd_lst(update_weights,weights_grad_tot,'+',learning_rate)
        update_biases=upd_lst(update_biases,biases_grad_tot,'+',learning_rate)
        # update_weights = update_weights + gamma*update_weights + learning_rate * weights_grad_tot
        # update_biases = update_biases + gamma*update_weight + learning_rate * biases_grad_tot
        weights=upd_lst(weights,update_weights,'-',1)
        biases=upd_lst(biases,update_biases,'-',1)
        num_data_point_done=0
        weights_grad_tot = [np.zeros(weights[i].shape) if(i != 0) else 0 for i in range(len(weights))]
        biases_grad_tot = [np.zeros(biases[i].shape) if(i != 0) else 0 for i in range(len(biases))]
  return [weights,biases]


  
def adagrad(X,Y,num_epochs,weight_decay,learning_rate,batch_size,weights,biases,activation_function,gamma,act_fn_derivative):
  #w_ahead=w_{t} - gamma. update_{t-1}
  #update{t} = gamma . update_{t-1} + learning_rate * grad(w_lookahead)
  # w_ahead= weights
  # b_ahead=biases
  for i in range(num_epochs):
    # num_hidden_layers=biases.size()[0] #Biases is a list of 1D arrays. Thus, the number of hidden layers is given by the size of this list
    Y_pred=feed_forward(X,activation_function,weights,biases) #Find prediction for each data instance of the data set
    weights_grad_tot=[]
    for w in weights:
      weights_grad_tot.append(np.zeroes(w.size()))
    biases_grad_tot=[]
    for b in biases:
      biases_grad_tot.append(np.zeroes(b.size()))
    num_data_point_done=0
    v_weight = 0 #Intitial value of 0
    v_bias=0
    epsilon = 0 #Figure out what is epsilon
    for x,y,y_pred in zip(X,Y,Y_pred):    
      [weights_grad,biases_grad] =  calculate_gradient(y,y_pred,h,a,weights,act_fn_derivative)
      weights_grad_tot = weights_grad_tot+weights_grad
      biases_grad_tot = biases_grad_tot + biases_grad
      num_data_point_done=num_data_point_done+1
      if(num_data_point_done==batch_size):
        v_weight = v_weight + (weights_grad_total)**2
        v_bias = v_bias + (biases_grad_total)**2
        v_weight_dum = learning_rate/np.sqrt(v_weight+epsilon)
        v_bias_dum = learning_rate/np.sqrt(v_bias+epsilon)
        weights=weights- v_weight_dum * weights_grad_tot
        biases=biases - v_bias_dum * bias_grad_tot
        num_data_point_done=0
    
def rmsprop(X,Y,num_epochs,weight_decay,learning_rate,batch_size,weights,biases,activation_function,gamma,act_fn_derivative):
  #w_ahead=w_{t} - gamma. update_{t-1}
  #update{t} = gamma . update_{t-1} + learning_rate * grad(w_lookahead)
  # w_ahead= weights
  # b_ahead=biases
  for i in range(num_epochs):
    num_hidden_layers=biases.size()[0] #Biases is a list of 1D arrays. Thus, the number of hidden layers is given by the size of this list
    Y_pred=feed_forward(X,activation_function,weights,biases) #Find prediction for each data instance of the data set
    weights_grad_tot=[]
    for w in weights:
      weights_grad_tot.append(np.zeroes(w.size()))
    biases_grad_tot=[]
    for b in biases:
      biases_grad_tot.append(np.zeroes(b.size()))
    num_data_point_done=0
    v_weight = 0 #Intitial value of 0
    v_bias=0
    epsilon = 0 #Figure out what is epsilon
    beta=0
    for x,y,y_pred in zip(X,Y,Y_pred):    
      [weights_grad,biases_grad] =  calculate_gradient(y,y_pred,h,a,weights,act_fn_derivative)
      weights_grad_tot = weights_grad_tot+weights_grad
      biases_grad_tot = biases_grad_tot + biases_grad
      num_data_point_done=num_data_point_done+1
      if(num_data_point_done==batch_size):
        v_weight = beta*v_weight + (1-beta)*(weights_grad_total)**2
        v_bias = beta*v_bias + (1-beta)*(biases_grad_total)**2
        v_weight_dum = learning_rate/np.sqrt(v_weight+epsilon)
        v_bias_dum = learning_rate/np.sqrt(v_bias+epsilon)
        weights=weights- v_weight_dum * weights_grad_tot
        biases=biases - v_bias_dum * bias_grad_tot
        num_data_point_done=0

In [ ]:
def backpropagation(X,Y,num_epochs,num_hidden_layers,sz_hidden_layer,weight_decay,learning_rate,optimizer,batch_size,weight_initialisation,activation_function,act_fn_derivative):
    
    
    #do the weights Initialisation here based on weight_initialisation,num_hidden_layers,sz_each_hidden_layer

    #bias is 0 initialised make that change
    
    weights = [0]  #dummy value
    biases =  [0]  #dummy value

    if (weight_initialisation == 'random'):
        
        num_classes = len(np.unique(Y))
        i = len(X[0].flatten())
        a = -1
        b = 1

        for j in range(num_hidden_layers-1):
            weights.append((b-a)*np.random.rand(sz_hidden_layer,i)+a)
            biases.append(np.zeros((sz_hidden_layer,1)))
            i = sz_hidden_layer

        weights.append((b-a)*np.random.rand(num_classes,i)+a)
        biases.append(np.zeros((num_classes,1)))

    else:

        #Xavier Implementation
        num_classes = len(np.unique(Y))
        i = len(X[0].flatten())

        for j in range(num_hidden_layers-1):
            t = math.sqrt(6)/math.sqrt(i + sz_hidden_layer)
            weights.append(np.random.uniform(-t,t,(sz_hidden_layer,i)))
            biases.append(np.zeros((sz_hidden_layer,1)))
            i = sz_hidden_layer
        
        t = math.sqrt(6)/math.sqrt(i + num_classes)
        weights.append(np.random.uniform(-t,t,(num_classes,i)))
        biases.append(np.zeros((num_classes,1)))




    if(optimizer == 'sgd'):
       return sgd(X,Y,num_epochs,weight_decay,learning_rate,batch_size,weights,biases,activation_function,act_fn_derivative)

    if(optimizer == 'momentum'): 
       return momentum(X,Y,num_epochs,weight_decay,learning_rate,batch_size,weights,biases,activation_function,act_fn_derivative)

    if(optimizer == 'nesterov'): 
      return nesterov(X,Y,num_epochs,weight_decay,learning_rate,batch_size,weights,biases,activation_function,act_fn_derivative)

    if(optimizer == 'rmsprop'):
      return rmsprop(X,Y,num_epochs,weight_decay,learning_rate,batch_size,weights,biases,activation_function,act_fn_derivative)

    if(optimizer == 'adam'):
      return adam(X,Y,num_epochs,weight_decay,learning_rate,batch_size,weights,biases,activation_function,act_fn_derivative)

    if(optimizer == 'nadam'):
      return nadam(X,Y,num_epochs,weight_decay,learning_rate,batch_size,weights,biases,activation_function,act_fn_derivative)



In [ ]:
def logistic(z):

  l = []
 # print(z.shape)
  for i in z:
     #print(type(i),i)
     if (i[0]<-600):
       l.append(0)
     else:
       l.append(1./(1+math.exp(-i[0])))
  
  t = np.array(l)

  t = t.reshape(t.shape[0],1)

  return t

def log_grad(z):
    return logistic(z)*(1-logistic(z))
def tanh(z):
    l = []
    for i in z:
      if (i[0]>=600):
        l.append(1)
      elif (i[0]<=(-600)):
        l.append(-1)
      else:
        l.append((math.exp(-i[0])-math.exp(-i[0]))/((math.exp(-i[0])+math.exp(-i[0]))))
        
    t = np.array(l)
    t = t.reshape(t.shape[0],1)
    return t

def tanh_grad(z):

  return 1-((tanh(z))*(tanh(z)))
def cross_entropy_loss(X_test,Y_test,weights,biases):

    sum = 0
    for i in range(Y_test.shape[0]):
      y_ = feed_forward(X_test[i],logistic,weights,biases)[2]
      if(Y_test[i]<y_.shape[0]):
        if(y_[Y_test[i]][0] != 0):
           sum = sum - math.log(y_[Y_test[i]][0])
        else:
           print("very bad\n")
           sum = sum + 100000

    return sum/Y_test.shape[0]
def accuracy(X_test,Y_test,weights,biases):
    
    sum = 0
    cnt = 0

    for i in range(Y_test.shape[0]):

     
      y_ = feed_forward(X_test[i],logistic,weights,biases)[2]
      y_ = y_.flatten()
      print(y_)
      j = np.argmax(y_)

      if (Y_test[i] == j):
        sum = sum+1

    return sum/Y_test.shape[0]

In [ ]:
import warnings
warnings.simplefilter('error', RuntimeWarning) # To help debug where the warnings are
#[weights,biases] = backpropagation(trainX,trainy,5,3,32,0,1e-3,'sgd',16,'random',logistic,log_grad)
[weights_momentum,biases_momentum] = backpropagation(trainX/255,trainy,5,3,32,0,1e-3,'momentum',16,'random',logistic,log_grad)

0
1
2
3
4


In [ ]:
#t = cross_entropy_loss(testX, testy,weights,biases)
t_momentum = cross_entropy_loss(testX, testy,weights_momentum,biases_momentum)

In [ ]:
#print(t)
print(t_momentum)

0.7287818550171089


In [ ]:
#t = accuracy(testX, testy,weights,biases)
t_momentum=accuracy(testX, testy,weights_momentum,biases_momentum)

Streaming output truncated to the last 5000 lines.
[2.83721423e-03 1.19855930e-03 4.99980741e-02 2.41239920e-03
 7.62289415e-03 9.10782037e-01 6.42719945e-03 1.58356395e-02
 3.31109436e-04 2.55487327e-03]
[6.70780144e-03 1.64378699e-02 6.51483966e-02 1.06691943e-02
 4.98389948e-01 1.77644543e-03 2.98109999e-01 7.43049968e-05
 1.02107894e-01 5.78145719e-04]
[3.81555182e-03 2.97418277e-06 6.07429798e-04 1.37895926e-04
 7.49564469e-05 1.17221156e-02 5.50871382e-04 9.73614079e-01
 4.64740940e-03 4.82671623e-03]
[8.62552144e-01 4.55612687e-03 2.31966991e-03 4.96781023e-02
 1.04545665e-03 4.51560350e-03 5.92693768e-02 9.62525670e-03
 6.00698335e-03 4.31280026e-04]
[1.06569140e-02 1.76671922e-03 3.59200026e-03 8.73699058e-01
 6.21100606e-02 1.07952888e-03 4.32323715e-02 1.47817980e-04
 2.68299761e-03 1.03253146e-03]
[8.43186909e-03 3.33698883e-02 1.45403237e-01 8.69498094e-02
 6.27930740e-01 4.35070820e-04 9.60675284e-02 5.88070994e-05
 1.19802607e-03 1.55023599e-04]
[1.04510759e-02 6.5369421

In [ ]:
#print(t)
print(t_momentum)

0.7425


In [ ]:
[a,b] = [1,2]

In [ ]:
[weights,biases] = backpropagation(trainX,trainy,5,3,32,0,1e-3,'sgd',16,'random',tanh,tanh_grad)

0
1
2
3
4


In [ ]:
t = cross_entropy_loss(testX, testy,weights,biases)
print(t)

1.3354825816019844


In [ ]:
t = accuracy(testX, testy,weights,biases)

Streaming output truncated to the last 5000 lines.
[5.36642488e-02 7.06013746e-01 4.57312642e-05 9.07207836e-03
 6.43170499e-03 1.18363886e-02 1.61864031e-04 2.11711725e-01
 2.91520948e-06 1.05959781e-03]
[9.76206208e-01 5.88510307e-04 5.76801489e-06 2.08706910e-03
 4.19210997e-05 3.56066897e-05 1.53496968e-04 1.98088955e-02
 1.59419790e-06 1.07092999e-03]
[6.37133221e-01 1.55792226e-01 1.20691160e-04 1.20286261e-02
 4.08365651e-03 3.60790772e-03 1.42099705e-04 1.77203844e-01
 9.98359258e-07 9.88673010e-03]
[4.74733014e-03 5.30770730e-02 3.57485593e-01 4.20104048e-03
 2.00256124e-01 6.08120260e-02 2.48316728e-01 5.63822832e-02
 1.45628276e-02 1.58975274e-04]
[5.36642488e-02 7.06013746e-01 4.57312642e-05 9.07207836e-03
 6.43170499e-03 1.18363886e-02 1.61864031e-04 2.11711725e-01
 2.91520948e-06 1.05959781e-03]
[3.47834704e-02 2.21046268e-01 2.64003735e-05 2.05575810e-03
 1.15883060e-03 3.15009368e-01 1.56164880e-03 1.07352343e-01
 3.28802597e-04 3.16677109e-01]
[9.76206208e-01 5.8851030

In [ ]:
print(t)

0.6177


In [ ]:
np.random.uniform(-5,5,(2,3))

array([[-1.6287139 ,  0.05543781,  3.30427875],
       [ 0.33914481, -0.34141066, -0.27107382]])

In [ ]:
[weights,biases] = backpropagation(trainX,trainy,5,3,32,0,1e-3,'sgd',16,'xavier',logistic,log_grad)

0
1
2
3
4


In [ ]:
t = cross_entropy_loss(testX, testy,weights,biases)
print(t)

1.2664653903162513


In [ ]:
t = accuracy(testX, testy,weights,biases)
print(t)

Streaming output truncated to the last 5000 lines.
 0.01337999 0.32931343 0.08656023 0.21213226]
[0.09161858 0.15359168 0.05271458 0.41556052 0.04727735 0.04580061
 0.055607   0.04951013 0.02144022 0.06687934]
[0.01586769 0.02060234 0.01229955 0.04856821 0.00869539 0.23633739
 0.01481331 0.12623962 0.0777623  0.43881419]
[0.14629645 0.20797476 0.04739728 0.32307438 0.04184599 0.03421576
 0.06171177 0.04799416 0.02282396 0.06666551]
[0.01188936 0.01325424 0.01158851 0.03497003 0.00801252 0.33665651
 0.01866898 0.24588994 0.10619628 0.21287362]
[0.09484588 0.0530673  0.19083782 0.08783525 0.22464194 0.02987102
 0.16366707 0.02870762 0.10285983 0.02366627]
[0.32402357 0.17198252 0.06933602 0.14043265 0.06531138 0.01443625
 0.13894813 0.0156137  0.03731816 0.02259763]
[0.15869115 0.06186482 0.14910901 0.11027033 0.13650304 0.03328331
 0.16639531 0.0347116  0.11220565 0.03696578]
[0.01834181 0.02027488 0.01769011 0.04657013 0.01421925 0.22956875
 0.02339433 0.09709068 0.11266044 0.42018963]

In [ ]:
[weights,biases] = backpropagation(trainX,trainy,5,3,32,0,1e-3,'sgd',16,'xavier',tanh,tanh_grad)

0
1
2
3
4


In [ ]:
t = cross_entropy_loss(testX, testy,weights,biases)
print(t)

1.423782205027862


In [ ]:
t = accuracy(testX, testy,weights,biases)
print(t)

Streaming output truncated to the last 5000 lines.
 0.03551874 0.34933573 0.1778851  0.08257446]
[0.05865365 0.06872047 0.15306001 0.57125593 0.01791829 0.02181188
 0.03878494 0.00971051 0.04869541 0.01138891]
[0.0089456  0.02493455 0.07294154 0.02465534 0.03128687 0.12819282
 0.07058469 0.37423415 0.0965113  0.16771314]
[0.09093042 0.09314473 0.21788849 0.40950888 0.02984287 0.02067677
 0.06743797 0.01262888 0.03807544 0.01986556]
[0.00372351 0.06492116 0.03422584 0.03892652 0.01146201 0.34174038
 0.0192352  0.24257434 0.20803843 0.03515261]
[0.1790249  0.06547364 0.27055995 0.19802005 0.04312243 0.01456214
 0.1307035  0.02003637 0.03344022 0.04505681]
[0.1941975  0.09431339 0.24610013 0.20797685 0.04982194 0.01674524
 0.11208559 0.02006655 0.02314342 0.03554939]
[0.07079454 0.0575687  0.21453159 0.46635464 0.02169221 0.01738174
 0.05074465 0.01182117 0.07090977 0.01820099]
[0.0089456  0.02493455 0.07294154 0.02465534 0.03128687 0.12819282
 0.07058469 0.37423415 0.0965113  0.16771314]